In [ ]:
from flask import Flask,render_template,request
import pickle  as pk
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

app=Flask(__name__)


@app.route("/")
def hari():
    return  render_template("hari.html")


@app.route("/predict",methods=["POST"])
def predict():
    if request.method=="POST":
        airline=str(request.form["Airline"])
        date=str(request.form["Date_of_Journey"])
        source=str(request.form["Source"])
        destination=str(request.form["Destination"])
        route=str(request.form["Route"])
        dep_time=str(request.form["Dep_Time"])
        arr_time=str(request.form["Arrival_Time"])
        duration=str(request.form["Duration"])
        total_stops=str(request.form["Total_Stops"])
        data=[[airline,date,source,destination,route,dep_time,arr_time,duration,total_stops]]
        user_data=pd.DataFrame(data,columns=["Airline","Date_of_Journey","Source","Destination","Route","Dep_Time","Arrival_Time","Duration","Total_Stops"])
        train=pd.read_excel("Data_Train.xlsx")
        train.dropna(inplace=True)
        train.drop(["Price","Additional_Info"],axis=1,inplace=True)
        test=pd.read_excel("Test_set.xlsx")
        test.drop(["Additional_Info"],axis=1,inplace=True)
        df=pd.concat([train,test,user_data],ignore_index=True)
        # we havet to convert (Date_of_journey,Dep_Time,Arrival_Time)from object to date time datatype
        for i in ["Date_of_Journey","Dep_Time","Arrival_Time"]:
            df[i]=pd.to_datetime(df[i])
        # we have to split the Date_of_journey columns as what date passenger travel what month passener travel
        df["journey_of_month"],df["journey_of_day"]=df["Date_of_Journey"].dt.month,df["Date_of_Journey"].dt.day
        # now we have to drop the date_of_journey column
        df.drop(["Date_of_Journey"],axis=1,inplace=True)
        #now we have to split the dep_time ,arrival_time columns as a what time(hour,minute) passenger is depature and arraval and drop that columns 
        for i in ["Dep_Time","Arrival_Time"]:
            df[i+"_hour"],df[i+"_minute"]=df[i].dt.hour,df[i].dt.minute
            df.drop(i,axis=1,inplace=True)
        # now we have to preprocess the duration column
        duration=list(df.Duration)
        for i,j in enumerate(duration):
            if len(j.split())==2:
                pass
            else:
                if "h"in j:
                    duration[i]=j+" 00m"
                else:
                    duration[i]="00h "+j
        df["Duration"]=duration
        df["Duration"+"_hour"]=df["Duration"].apply(lambda x:int(x.split()[0][:-1]))
        df["Duration"+"_minute"]=df["Duration"].apply(lambda x:int(x.split()[1][:-1]))
        df.drop("Duration",axis=1,inplace=True)
        for i in df.columns:
            if df[i].dtype=="O":
                df[i]=df[i].astype("category")
        
        category=df.select_dtypes(include="category")
        numeric=df.select_dtypes(include="int64")
        for i in range(5):
            category["Route_"+str(i+1)]=category["Route"].str.split("→").str[i]
        for i in category.columns[-5:]:
            category[i]=category[i].astype(str)
            category[i]=category[i].astype("category")
        category.replace("nan","None",inplace=True)
        category.drop(["Route"],axis=1,inplace=True)
        category.replace({"Delhi":"New Delhi"},inplace=True)
        le=LabelEncoder()
        for i in category.columns[-5:]:
               category[i]=le.fit_transform(category[i])
        s=list(category["Source"].unique())
        d=list(category["Destination"].unique())
        s.extend(d)
        for i in s:
            category[i]=np.where(((category["Source"]==i)|(category["Destination"]==i)),1,0)
        category.drop(["Destination","Source"],axis=1,inplace=True)
        #we have to preprocess the Total_stops column
        category["Total_Stops"]=category["Total_Stops"].apply(lambda s:0 if s=="non-stop" else int(s[0])).astype("int64")
        category=pd.get_dummies(category)
        c_df=pd.concat([category,numeric],axis=1) 
        with open("pr_model","rb") as f:
            m=pk.load(f)
        user_input=np.array(c_df.iloc[-1,:]).reshape(1,-1)
        value=m.predict(user_input)[0]
    return render_template("hari.html",prediction=value)

if __name__=="__main__":
    app.run()
        

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jul/2021 20:53:34] "GET / HTTP/1.1" 200 -
[2021-07-05 20:57:52,486] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoin

127.0.0.1 - - [05/Jul/2021 21:03:19] "POST /predict HTTP/1.1" 200 -


In [ ]:
t=[]


In [ ]:
t.e